###  Data replot 


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import mpl_toolkits.axes_grid1
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
from PIL import Image


from qfit import make_angle_file as maf
from qfit import fit_q
from qfit import re_analysis as rean
from qfit import re_plot as replt
from qfit import multiplot as mlplt
from qfit import image_treat as imt
from qfit import file_folder_trans as fft

In [ ]:
import warnings
warnings.simplefilter('ignore')
# warnings.resetwarnings()
# warnings.simplefilter('ignore', FutureWarning)
%load_ext autoreload
%autoreload 2

In [ ]:
# plt.rcParams["font.size"] = 18
# plt.rcParams['font.family']= 'sans-serif'
# plt.rcParams['font.sans-serif'] = ['Arial']

In [ ]:
# RC triming and ratation files
# rot_tr=['gauss_fit/rot_tr_gaussian_220614_100214','gauss_fit/rot_tr_gaussian_220614_110511','gauss_fit/rot_tr_gaussian_220614_104721']
rot_tr=[r'',r'',]

In [ ]:
# load RC Plot
# return -> {'c':peak original, 'h':height original , 'w':width original (FWHM), 
#            'ct':peak - average [deg], 'ht':(height - average) /average, 'wt':width [deg] }  
# load data include nan value.  If calculation of max, min and so on,   remove nan valu. 
# example: tmp_ave = np.mean(tmp_array[~np.isnan(tmp_array)]) or tmp_ave = np.nanmean(tmp_array)

rc0 = rean.load_rc_tif(rot_tr[0])
rcp120 = rean.load_rc_tif(rot_tr[1])
rcm120 = rean.load_rc_tif(rot_tr[2])

In [ ]:
# rc plot
mlplt.rc_2plots(rc0, title='$\psi$ = 0$^{\circ}$', peak_range=(-0.3,0.3), width_range=(0,0.05), save=False)
mlplt.rc_3plots(rc0, title='$\psi$ = 0$^{\circ}$',peak_range=(-0.3,0.3), width_range=(0,0.05), save=False)

In [ ]:
# radial quality evaluation
from qfit import radial_average as rda

In [ ]:
# find center and radius using height image of RC
# create binary image astype uint8 
b_h_img = rda.image2uint8(rc0['h'])

# 4inch area size-> 2500000, 2inch -> 60000, at pixel size 0.05mm
hc_lists, hr_lists, ha_lists = rda.find_circle(b_image=b_h_img, area_size=4000)

In [ ]:
# phi=0
r0_dsi, r0_dri = rda.calc_circle_radius_ave(org_image=rc0['wt'], center=hc_lists[0], max_radius=hr_lists[0], 
                                            dr=100, pixel_size=0.05, fig_show=True, ylim=(0.01,0.02))

In [ ]:
# b_h_img = rda.image2uint8(rc0['h'])
# hc_lists, hr_lists, ha_lists = rda.find_circle(b_image=b_h_img, area_size=400000)

In [ ]:
rm120dsi, rm120dri = rda.calc_circle_radius_ave(org_image=rcm120['wt'], center=hc_lists[0], max_radius=hr_lists[0], 
                                                dr=100, pixel_size=0.05, fig_show=True, ylim=(0.01,0.02))

In [ ]:

rp120dsi, rp120dri = rda.calc_circle_radius_ave(org_image=rcp120['wt'], center=hc_lists[0], max_radius=hr_lists[0], 
                                                dr=100, pixel_size=0.05, fig_show=True, ylim=(0.01,0.02))
                                              

In [ ]:
rda.r_dr_3data_plot(r0_dsi,r0_dri,rp120dsi,rp120dri,rm120dsi,rm120dri,
                    ylim=(0.01,0.02),
                    title='r $\cdot$ $\Delta r$',
                    save=True)

In [ ]:
# Limit estimation

In [ ]:
from qfit import d_limit as dlim
gan_param = {"ramda":1.284, 'd':1.004, 'dp':0.05,'L':500, 'drr':2.8e-04}

dlim.limit_estimate(**gan_param)

In [ ]:
dlim.dd_calc_4plot(data=rc0, title='$\Delta$ d/d at $\psi=0$', dd_xange=(0,2.5e-4),**gan_param)
dlim.dpsi_calc_4plot(data=rc0, title='$\Delta \Psi$ at $\psi=0$',dp_xange=(0,6e-3),**gan_param )
dlim.limt_calc_4plot(data=rc0, title='Limit $\Delta$d/d $\cdot$ $\Delta\Psi$ at $\psi=0$',dd_xange=(0,2.5e-4),dp_xange=(0,6e-3),**gan_param,save=True)

In [ ]:
### Q replot

In [ ]:
# q_data =[r'gauss_fit\2R_0m120',r'gauss_fit\2R_0p120',r'gauss_fit\2R_pm120']
q_data =[r'',r'',r'']

In [ ]:
# load q 
# dict : {'x':qx, 'y':qy, 'z':qz, 'xy':qxy, 'ang':q_ang, 'angxy':q_angxy, 'r':q_r}
# q_ang: polar codinate angle between Z and r  -> theta
# q_angxy: polar codinate angle between x and y -> phi

q_0p = rean.load_q_tif(q_data[0])
q_0m = rean.load_q_tif(q_data[1])
q_pm = rean.load_q_tif(q_data[2])

In [ ]:
mlplt.qc_4plots(q_0m, title='q$_x$, q$_y$ at (0,m120)',qx_range=(-0.04,0.04),qy_range=(-0.04,0.04))
mlplt.qc_4plots(q_0p, title='q$_x$, q$_y$ at (0,p120)',qx_range=(-0.04,0.04),qy_range=(-0.04,0.04))
mlplt.qc_4plots(q_pm, title='q$_x$, q$_y$ at (p120,m120)',qx_range=(-0.04,0.04),qy_range=(-0.04,0.04))

In [ ]:
mlplt.qp3_4plots(q_0m, title='$q_r$,$q_{\\theta}$,$q_{\phi}$', step=150)

In [ ]:
# average q
ave_q= rean.average_3q(q_0p, q_0m, q_pm)


In [ ]:
mlplt.qc_4plots(ave_q, title='q$_x$, q$_y$ average',qx_range=(-0.03,0.03),qy_range=(-0.03,0.03),save=True)
mlplt.qp3_4plots(ave_q, title='$q_r$,$q_{\\theta}$,$q_{\phi}$ average', step=150,save=True)

In [ ]:
mlplt.qp_4plots(ave_q, title='$q_{\\theta}$$q_{\phi}$')
mlplt.qp3_3plots(ave_q, title='$q_{\phi}$,$q_{xy}$, q arrow', step=150)

In [ ]:
mlplt.qcp_12plots(ave_q, title='q average', step=100)

In [ ]:
# histgram
# The data has nan values.

xyzdata =rc0['wt']
xyzdata = xyzdata.reshape(-1)
xyzdata = xyzdata[~np.isnan(xyzdata)]
bins = np.linspace(0, 0.1, 300)

print(np.mean(xyzdata))
print(np.max(xyzdata))

print(np.min(xyzdata))
print(np.percentile(xyzdata,50))
print(np.percentile(xyzdata,60))
rdata, rbins = np.histogram(xyzdata, bins=bins)

# print(rdata,rbins)
plt.plot(rbins[:-1],rdata)
print(rbins[np.argmax(rdata)])
print()

In [ ]:
qxyz =ave_q['ang']
qxyz = qxyz.reshape(-1)
qxyz = qxyz[~np.isnan(qxyz)]
bins = np.linspace(0, 0.3, 100)
qrdata, qrbins = np.histogram(qxyz, bins=bins)

# print(rdata,rbins)
plt.plot(qrbins[:-1],qrdata)
print(rbins[np.argmax(qrdata)])
print(np.max(qrdata))

print(np.mean(qxyz))
print(np.min(qxyz))
print(np.percentile(qxyz,50))
print(np.percentile(qxyz,60))